In [1]:
import datetime, itertools, sys
import pandas as ps
from subprocess import Popen, PIPE

In [2]:
data = ps.read_csv("data/spelled-f.csv", sep=';', header=None,
                   index_col=0,names=['id','title','text','cluster','date','publisher'])

## Предварительная очистка данных

In [3]:
data = data[~data["cluster"].isin(["-", "S", "Standard "])]
print("Число записей в таблице:", len(data))

Число записей в таблице: 32317


## Приведение к нормальной форме и определение граммемы

In [4]:
MYSTEM_GRAMMEMS = ["NAME", "A", "ADV", "ADVPRO", "ANUM", "APRO", "COM",
    "CONJ", "INTJ", "NUM", "PART", "PR", "S", "SPRO", "V"]
def mystem_parse(texts):
    '''
        Прогнать список текстов через mystem, вернув для каждого слова
        его нормальную форму и граммему.
        Аргументы:
            texts - список строковых значений;
        Возвращает: список наборов, где каждый набор соответствует
        входному тексту и содержит кортежи (нормальная форма, граммема)
    '''
    result = []
    # В качестве разделителя сообщений используем волшебное слово.
    # Грязно, но работает:
    text = "\nDEADBEEF\n".join(texts)

    print("Calling mystem...", end=' ')
    sys.stdout.flush()
    pipe = Popen(["mystem", "-lni"], stdout=PIPE, stdin=PIPE)
    raw = pipe.communicate(text.encode("utf-8"))[0].decode("utf-8")
    print("Done.")

    msg = []
    for line in raw.split():
        if "DEADBEEF" in line:
            result.append(msg)
            msg = []
            continue
        if line[-1] == '?':
            # Если mystem не опознал слово:
            norm = line
            gramm = "NAME"
        else:
            norm = line[:line.find("=")]
            gramm = None
            line = line.split("|")[0][line.find("=")+1:]
            for trait in ("гео", "имя", "фам", "отч"):
                if trait in line:
                    gramm = "NAME"
                    break
            if gramm is None:
                gramm = line.split('=')[0].split(',')[0]
        assert gramm in MYSTEM_GRAMMEMS
        norm = norm.strip('?')
        msg.append((norm, gramm))
    result.append(msg)
    return result

In [5]:
texts = [row["title"] + ". " + row["text"] for ind, row in data.iterrows()]
parsed = mystem_parse(texts)

Calling mystem... Done.


In [6]:
messages = {}
base_date = datetime.datetime.strptime(min(data["date"]), "%Y-%m-%d").date()
for stem, (ind, row) in zip(parsed, data.iterrows()):
    date = datetime.datetime.strptime(row["date"], "%Y-%m-%d").date()
    stamp = (date - base_date).days
    messages[ind] = {"stem": stem, "date": stamp, "publisher": row["publisher"], "cluster": row["cluster"]}
print("Число сообщений:", len(messages))

Число сообщений: 32317


## Составляем список пар для классификации

In [7]:
intersections = {}
allowed_grammems = set(["NAME", "A", "ADJ", "V"])
for id, msg in messages.items():
    for st in msg["stem"]:
        if st[1] not in allowed_grammems or len(st[0]) < 4: continue
        if st not in intersections:
            intersections[st] = set()
        intersections[st].add(id)
print("Максимальный кластер:", max(len(val) for key, val in intersections.items()))

Максимальный кластер: 4381


In [8]:
day_intersections = {}
for st, int_list in intersections.items():
    for msg_id in int_list:
        msg = messages[msg_id]
        for day in range(msg["date"], msg["date"]+2):
            key = (st, day)
            if key not in day_intersections:
                day_intersections[key] = set()
            day_intersections[key].add(msg_id)

day_intersections = {key: value for key, value in day_intersections.items() if len(value) > 1}
ln, key = max((len(val), key) for key, val in day_intersections.items())
print("Максимальный кластер:", ln)
print("Число кластеров:", len(day_intersections))
print("Кластеры > 500:", sorted(len(cluster) for key, cluster in day_intersections.items() if len(cluster) > 500))

Максимальный кластер: 2995
Число кластеров: 50378
Кластеры > 500: [503, 505, 508, 513, 513, 525, 529, 535, 541, 543, 546, 550, 557, 557, 561, 564, 565, 572, 579, 586, 587, 593, 596, 597, 601, 606, 611, 617, 627, 632, 637, 639, 642, 644, 655, 749, 783, 818, 829, 830, 843, 877, 924, 957, 983, 1010, 1021, 1046, 1091, 1133, 1138, 1499, 1583, 1611, 1736, 1737, 1762, 1831, 1935, 2299, 2402, 2826, 2858, 2869, 2887, 2951, 2995]


In [14]:
pairs = 0
for cluster in day_intersections.values():
    #if len(cluster) > 100: continue
    pairs += len(cluster) * (len(cluster)-1)//2
print("Максимум пар для классификации:", pairs)

Максимум пар для классификации: 76534703


In [15]:
pairs = set()
for cluster in day_intersections.values():
    for id1, id2 in itertools.combinations(cluster, 2):
        if messages[id1]["cluster"] == messages[id2]["cluster"]:
            pairs.add(frozenset((id1, id2)))
print("Число пар семантических дубликатов:", len(pairs))

Число пар семантических дубликатов: 24241


In [23]:
key_dups = {}
for key, cluster in day_intersections.items():
    for id1, id2 in itertools.combinations(cluster, 2):
        if messages[id1]["cluster"] == messages[id2]["cluster"]:
            key_dups[key] = key_dups.get(key, 0) + 1
print(sorted((val, key, len(day_intersections[key])) for key, val in key_dups.items())[-1:-20:-1])

[(1336, (('россия', 'NAME'), 370), 2951), (1305, (('россия', 'NAME'), 371), 2887), (1211, (('украина', 'NAME'), 371), 2402), (1162, (('украина', 'NAME'), 370), 2299), (797, (('новый', 'A'), 370), 1935), (609, (('сообщать', 'V'), 371), 2995), (582, (('сообщать', 'V'), 370), 2869), (579, (('новый', 'A'), 371), 1499), (574, (('российский', 'A'), 371), 1762), (568, (('быть', 'V'), 370), 2858), (561, (('российский', 'A'), 370), 1737), (560, (('быть', 'V'), 371), 2826), (427, (('заявлять', 'V'), 371), 1831), (425, (('москва', 'NAME'), 371), 830), (419, (('москва', 'NAME'), 370), 818), (399, (('заявлять', 'V'), 370), 1736), (385, (('украинский', 'NAME'), 371), 1091), (368, (('украинский', 'NAME'), 370), 1046), (322, (('главный', 'A'), 371), 1010)]


## Создание таблиц признаков

In [9]:
messages_sets = {key: set(msg["stem"]) for key, msg in messages.items()}

def make_feature_table(cluster):
    rows = []
    for id1, id2 in itertools.combinations(cluster, 2):
        if messages[id1]["publisher"] == messages[id2]["publisher"]:
            continue
        common = get_common(messages_sets[id1], messages_sets[id2])
        rows.append( tuple(sorted((id1, id2)))
                    + tuple(common.get(col, 0) for col in MYSTEM_GRAMMEMS)
                    + (int(messages[id1]["cluster"] == messages[id2]["cluster"]),)
                   )
    return ps.DataFrame(rows, columns=["id1", "id2"]+MYSTEM_GRAMMEMS+["is_dup"])

def get_common(m1, m2):
    common = {}
    for norm, grammem in (m1 & m2):
        common[grammem] = common.get(grammem, 0) + 1
    return common

In [14]:
tables = []
total = sum(1 for cluster in day_intersections.values() if len(cluster) <= 25)
cur = 0
for cluster in day_intersections.values():
    if len(cluster) > 25: continue
    tables.append(make_feature_table(cluster))
    cur += 1
print("Готово.            ")
df = ps.concat(tables)
df.to_csv("data/table-cluster-le25.csv", index=False)

Готово.            


In [15]:
df2 = df.drop_duplicates()
print(len(df[df["is_dup"] == 1]))
print(len(df2[df2["is_dup"] == 1]))

73520
17685


In [16]:
df2.to_csv("data/table-cluster-le25.csv", index=False)

## Создание таблицы одного кластера

In [24]:
cls_key = (('москва', 'NAME'), 371)
print(len(day_intersections[cls_key]))
df = make_feature_table(day_intersections[cls_key])
df.to_csv("data/table-cluster-москва-371.csv")

830
